In [1]:
from sklearnex import patch_sklearn
import sys
import os
# SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
# sys.path.append(os.path.dirname(SCRIPT_DIR))
patch_sklearn()
sys.path.append("..")

from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from imblearn.over_sampling import SMOTE
from models.mlmodels import KNN_model_predict, LDA_model_predict, SVM_model_predict, XGB_model_predict, KNNDWN_model_predict
from utils.plots import get_metrics
from utils.dataloader import load_eeg_data_edf, load_eeg_data_gdf, load_eeg_data_mat
from utils.data_preprocessing_utils import Utils
from utils.feature_extraction import time_domain_features, standard_scaler_transform, wavelet_transform, bandpass_transform, psd_transform



def run_ml_models(dataset, params, metrics_results):

    # ------------------------- KNN ---------------------------------------------------
    y_true, y_pred, best_score = KNN_model_predict(dataset['X_train'],
                                                   dataset['X_test'],
                                                   dataset['Y_train'],
                                                   dataset['Y_test'],
                                                   scoring=None)
    metrics_results.append(get_metrics(
        y_true, y_pred, best_score, 'KNN', params))

    # ------------------------- SVM ---------------------------------------------------
    y_true, y_pred, best_score = SVM_model_predict(dataset['X_train'],
                                                   dataset['X_test'],
                                                   dataset['Y_train'],
                                                   dataset['Y_test'],
                                                   scoring=None)
    metrics_results.append(get_metrics(
        y_true, y_pred, best_score, 'SVM', params))

    # ------------------------- XGB ---------------------------------------------------
    y_true, y_pred, best_score = XGB_model_predict(dataset['X_train'],
                                                   dataset['X_test'],
                                                   dataset['Y_train'],
                                                   dataset['Y_test'],
                                                   scoring=None)
    metrics_results.append(get_metrics(
        y_true, y_pred, best_score, 'XGB', params))


SOURCE_PATH = "C:/Users/Maods/Documents/Code-Samples/Python/MI-EEG-Dataset/dataset/processed"

# Load data
# ["FC1", "FC2"], ["FC3", "FC4"], ["FC5", "FC6"]]
channels = Utils.combinations["e"]

exclude = [38, 88, 89, 92, 100, 104]
subjects = [n for n in np.arange(1, 110) if n not in exclude]

x, y = Utils.load(channels, subjects, base_path=SOURCE_PATH)
y = Utils.to_categorical(y)


# Reshape for scaling
x_reshaped = x.reshape(x.shape[0], x.shape[1] * x.shape[2])
# Grab a test set before SMOTE
x_train, x_test, y_train, y_test = train_test_split(x_reshaped,
                                                    y,
                                                    stratify=y,
                                                    test_size=0.20,
                                                    random_state=42)

# Scale indipendently train/test
x_train_scaled = minmax_scale(x_train, axis=1)
x_test_scaled = minmax_scale(x_test, axis=1)


# apply smote to train data
print('classes count')
print(f'before oversampling = {y_train.sum(axis=0)}')

# smote
sm = SMOTE(random_state=42)
x_train_smote, y_train = sm.fit_resample(x_train_scaled, y_train)
print('classes count')
print(f'after oversampling = {y_train.sum(axis=0)}')


# Reshape for original format (x, 640, 2)
x_train = x_train_smote.reshape(x_train_smote.shape[0], int( x_train_smote.shape[1]/2), 2).astype(np.float64)
x_test = x_test_scaled.reshape(x_test_scaled.shape[0], int( x_test_scaled.shape[1]/2), 2).astype(np.float64)


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.

classes count
before oversampling = 110811
classes count
after oversampling = 444960


In [2]:
metrics_results = []


# ===> Feture Extraction A
params = dict(
    fs=160,
    dataset='Motor Imaginary',
    subject='All subjects',
    pipeline='time domain features -> wavelet'
)
features_a_train = wavelet_transform(x_train, params)
features_a_test = wavelet_transform(x_test, params)

c:\Users\Maods\Documents\Repos\EEG-Analysis-\env_tcc_eeg\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [3]:
features_a_train = time_domain_features(features_a_train, params['fs'])
features_a_test = time_domain_features(features_a_test, params['fs'])

c:\Users\Maods\Documents\Repos\EEG-Analysis-\env_tcc_eeg\lib\site-packages\scipy\signal\_spectral_py.py:1961: UserWarning: nperseg = 256 is greater than input length  = 4, using nperseg = 4
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\Maods\Documents\Repos\EEG-Analysis-\env_tcc_eeg\lib\site-packages\antropy\entropy.py:248: RuntimeWarning: invalid value encountered in true_divide
  psd_norm = psd / psd.sum(axis=axis, keepdims=True)
c:\Users\Maods\Documents\Repos\EEG-Analysis-\env_tcc_eeg\lib\site-packages\antropy\entropy.py:249: RuntimeWarning: divide by zero encountered in log2
  se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)
c:\Users\Maods\Documents\Repos\EEG-Analysis-\env_tcc_eeg\lib\site-packages\antropy\entropy.py:249: RuntimeWarning: invalid value encountered in multiply
  se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)


In [4]:
import gc

del x_train_scaled
del x_test_scaled
del x_reshaped

del x
del y

del x_train_smote

gc.collect()

319

In [11]:
features_a_train.nbytes 

9112780800

In [14]:
features_a_train.nbytes / 1e+9

9.1127808

In [15]:
a = features_a_train.astype(np.float32)
a.nbytes / 1e+9

4.5563904

In [5]:
dataset = dict(
    X_train=features_a_train,
    X_test=features_a_test,
    Y_train=y_train,
    Y_test=y_test
)


run_ml_models(dataset, params, metrics_results)



KeyboardInterrupt: 